In [1]:
import pandas as pd
from datetime import datetime, timedelta

start_date = datetime(2018,1,1)
end_date = datetime(2022,12,31)

year_list = []
month_list = []
day_list = []
date_list = []

current_date = start_date
while current_date <= end_date:
  year_list.append(current_date.year)
  month_list.append(current_date.month)
  day_list.append(current_date.day)
  date_list.append(current_date.strftime('%Y-%m-%d'))
  current_date += timedelta(days=1)

date_df = pd.DataFrame({
  'Year': year_list,
  'Month': month_list,
  'Day': day_list,
  'Date': date_list,
})

In [11]:
date_df

,Year,Month,Day,Date
0,2018,1,1,2018-01-01
1,2018,1,2,2018-01-02
2,2018,1,3,2018-01-03
3,2018,1,4,2018-01-04
4,2018,1,5,2018-01-05
...,...,...,...,...
1821,2022,12,27,2022-12-27
1822,2022,12,28,2022-12-28
1823,2022,12,29,2022-12-29
1824,2022,12,30,2022-12-30


In [12]:
import json
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as BS
import os
import glob
from tqdm import tqdm

In [13]:
start_row = 1
end_row = 1000

data_list = []
while True:
  api_key = ''
  url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/tbCycleStationInfo/{start_row}/{end_row}'
  r = requests.get(url)
  data = r.json()
  data_list.append(data)
  if 'INFO-200' in r.text: 
    break
  start_row += 1000
  end_row += 1000
  print(url)

http://openapi.seoul.go.kr:8088/48587572526c6f7633345a51726562/json/tbCycleStationInfo/1/1000
http://openapi.seoul.go.kr:8088/48587572526c6f7633345a51726562/json/tbCycleStationInfo/1001/2000
http://openapi.seoul.go.kr:8088/48587572526c6f7633345a51726562/json/tbCycleStationInfo/2001/3000
http://openapi.seoul.go.kr:8088/48587572526c6f7633345a51726562/json/tbCycleStationInfo/3001/4000


In [14]:
data_list

[{'stationInfo': {'list_total_count': '3079',
   'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
   'row': [{'STA_LOC': '마포구',
     'RENT_ID': 'ST-10',
     'RENT_NO': '00108',
     'RENT_NM': '서교동 사거리',
     'RENT_ID_NM': '108. 서교동 사거리',
     'HOLD_NUM': '12',
     'STA_ADD1': '서울특별시 마포구 양화로 93',
     'STA_ADD2': '427',
     'STA_LAT': '37.55274582',
     'STA_LONG': '126.91861725',
     'START_INDEX': 0,
     'END_INDEX': 0,
     'RNUM': '1'},
    {'STA_LOC': '광진구',
     'RENT_ID': 'ST-100',
     'RENT_NO': '00503',
     'RENT_NM': '더샵스타시티 C동 앞',
     'RENT_ID_NM': '503. 더샵스타시티 C동 앞',
     'HOLD_NUM': '15',
     'STA_ADD1': '서울특별시 광진구 아차산로 262',
     'STA_ADD2': '더샵스타시티 C동 앞',
     'STA_LAT': '37.53666687',
     'STA_LONG': '127.07359314',
     'START_INDEX': 0,
     'END_INDEX': 0,
     'RNUM': '2'},
    {'STA_LOC': '양천구',
     'RENT_ID': 'ST-1000',
     'RENT_NO': '00729',
     'RENT_NM': '서부식자재마트 건너편',
     'RENT_ID_NM': '729. 서부식자재마트 건너편',
     'HOLD_NUM': '10',
     'S

In [15]:
dfs = []
for i in range(0,len(data_list)-1):
  df = pd.DataFrame(data_list[i]['stationInfo']['row'])
  dfs.append(df)
  print(df)
  
hire_df = pd.concat(dfs, ignore_index=True)
print(hire_df)


    STA_LOC  RENT_ID RENT_NO          RENT_NM           RENT_ID_NM HOLD_NUM  \
0       마포구    ST-10   00108          서교동 사거리         108. 서교동 사거리       12   
1       광진구   ST-100   00503      더샵스타시티 C동 앞     503. 더샵스타시티 C동 앞       15   
2       양천구  ST-1000   00729      서부식자재마트 건너편     729. 서부식자재마트 건너편       10   
3       양천구  ST-1002   00731     서울시 도로환경관리센터    731. 서울시 도로환경관리센터       10   
4       양천구  ST-1003   00732            신월중학교           732. 신월중학교       10   
..      ...      ...     ...              ...                  ...      ...   
995     양천구  ST-1997   00771     목동아파트 703동 앞     771.목동아파트 703동 앞        8   
996     양천구  ST-1998   00772       대원칸타빌1차아파트       772.대원칸타빌1차아파트       15   
997     양천구  ST-1999   00773  염창역 3, 4번 출구 사이  773.염창역 3, 4번 출구 사이       10   
998     마포구    ST-20   00114     홍대입구역 8번출구 앞    114. 홍대입구역 8번출구 앞       15   
999     마포구   ST-200   00142       아현역 4번출구 앞      142. 아현역 4번출구 앞       11   

                           STA_ADD1      STA_ADD2  

In [16]:
hire_df

,STA_LOC,RENT_ID,RENT_NO,RENT_NM,RENT_ID_NM,HOLD_NUM,STA_ADD1,STA_ADD2,STA_LAT,STA_LONG,START_INDEX,END_INDEX,RNUM
0,마포구,ST-10,00108,서교동 사거리,108. 서교동 사거리,12,서울특별시 마포구 양화로 93,427,37.55274582,126.91861725,0,0,1
1,광진구,ST-100,00503,더샵스타시티 C동 앞,503. 더샵스타시티 C동 앞,15,서울특별시 광진구 아차산로 262,더샵스타시티 C동 앞,37.53666687,127.07359314,0,0,2
2,양천구,ST-1000,00729,서부식자재마트 건너편,729. 서부식자재마트 건너편,10,서울특별시 양천구 신정동 236,서부식자재마트 건너편,37.51037979,126.86679840,0,0,3
3,양천구,ST-1002,00731,서울시 도로환경관리센터,731. 서울시 도로환경관리센터,10,서울특별시 양천구 목동동로 316-6,서울시 도로환경관리센터,37.52989960,126.87654114,0,0,4
4,양천구,ST-1003,00732,신월중학교,732. 신월중학교,10,서울특별시 양천구 화곡로 59,신월동 이마트,37.53955078,126.82830048,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3074,양천구,ST-993,00722,경서농협 버스정류장(우리은행신정동지점방향),722. 경서농협 버스정류장(우리은행신정동지점방향),10,서울특별시 양천구 신월로 342-1 구두수선대19,,37.52151108,126.85738373,0,0,3075
3075,양천구,ST-994,00723,SBS방송국,723. SBS방송국,15,서울특별시 양천구 목동서로161,SBS방송국,37.52916336,126.87274933,0,0,3076
3076,양천구,ST-995,00724,계남공원 입구 주출입구 좌측,724. 계남공원 입구 주출입구 좌측,10,서울특별시 양천구 중앙로 153 공중화장실,,37.51059723,126.85732269,0,0,3077
3077,양천구,ST-996,00725,양강중학교앞 교차로,725. 양강중학교앞 교차로,15,서울특별시 양천구 남부순환로88길5-16,양강중학교앞 교차로,37.52433395,126.85054779,0,0,3078


In [17]:
hire_df.to_csv('hire_point.csv')

In [ ]:
pd.read_csv('hire_point.csv')